In [ ]:
# !pip install optuna

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import sys
# Append the directory of the script to sys.path
sys.path.append("../src/ITM_Classifier-baseline")

# Now you can import the classes
from ITM_Classifier_baseline import ITM_DataLoader, ITM_Classifier

In [ ]:
def create_model(trial):
    # Define hyperparameters
    num_projection_layers = trial.suggest_int('num_projection_layers', 1, 3)
    projection_dims = trial.suggest_int('projection_dims', 64, 256)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)

    # Initialize the ITM Classifier with the suggested hyperparameters
    itm_classifier = ITM_Classifier(
        num_projection_layers=num_projection_layers, 
        projection_dims=projection_dims, 
        dropout_rate=dropout_rate, 
        learning_rate=learning_rate
    )
    return itm_classifier

def objective(trial):
    # Create and train the model
    itm_classifier = create_model(trial)
    
    # Assuming that the `test_classifier_model` method returns the test accuracy
    # Modify your ITM_Classifier to return accuracy from `test_classifier_model` if not already
    accuracy = itm_classifier.test_classifier_model()
    
    # Return the negative accuracy since Optuna minimizes the objective
    return 1 - accuracy


In [ ]:
def main():
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=20)  # Set the number of trials based on your resource availability

    print('Best trial:')
    trial = study.best_trial

    print(f'  Value: {1 - trial.value}')
    print('  Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

In [ ]:
if __name__ == "__main__":
    main()